In [1]:
import pandas as pd
import numpy as np
import altair as alt
from scipy.stats.mstats import gmean

In [2]:
stocks = pd.read_csv('clean.csv')
index = pd.read_csv('yahoo.csv')
arr = np.array(stocks)[:,1:]

In [3]:
def nth_worst_stock_every_n_days(data, n=0, n_days=1, offset=0):
    returns = [1]*offset
    money = 1
    for i in range(offset + 1, len(data) - n_days - 1, n_days):
        worst = nth_worst_stock(data, i, n)
        buy = data[i][worst]
        sell = data[i + n_days][worst]
        money *= sell/buy
        returns.extend([money]*n_days)
    while len(returns) < len(data):
        returns.append(money)
    return returns

def nth_worst_stock(data, day, n):
    gains = data[day]/data[day - 1]
    a = np.argsort(gains)
    return a[n]

In [4]:
ours = nth_worst_stock_every_n_days(arr, 9, 3, 1)
theirs = np.array(index['Adj Close'])/index['Adj Close'][0]
dates = np.array(stocks['Date'])
results = pd.DataFrame({'Date': dates, 'Ours': ours, 'S&P500': theirs})

In [5]:
alt.Chart(results).mark_line(
    color='blue'
).encode(
    alt.X('Date:T'),
    alt.Y('S&P500:Q', axis=alt.Axis(format='%'))
) + alt.Chart(results).mark_line(
    color='red'
).encode(
    alt.X('Date:T'),
    alt.Y('Ours:Q', axis=alt.Axis(format='%'))
)

alt.LayerChart(...)

In [6]:
alt.Chart(results).mark_line(
    color='blue'
).encode(
    alt.X('Date:T'),
    alt.Y('S&P500:Q',
          axis=alt.Axis(format='%'),
          scale=alt.Scale(type='log'))
) + alt.Chart(results).mark_line(
    color='red'
).encode(
    alt.X('Date:T'),
    alt.Y('Ours:Q',
          axis=alt.Axis(format='%'),
          scale=alt.Scale(type='log'))
)

alt.LayerChart(...)

In [7]:
combined_return = np.concatenate((ours, theirs))
combined_strategy = ['Our Strategy']*3021 + ['S&P 500']*3021
combined_dates = np.concatenate((dates, dates))
combined = pd.DataFrame({'Date': combined_dates,
                         'Strategy': combined_strategy,
                         'Return': combined_return})

In [8]:
alt.data_transformers.disable_max_rows()

alt.Chart(combined).mark_line().encode(
    alt.X('Date:T'),
    alt.Y('Return:Q', axis=alt.Axis(format='%')),
    alt.Color('Strategy:O'),
    tooltip=['Date:T', 'Return:Q', 'Strategy:O']
)

alt.Chart(...)

In [9]:
alt.Chart(combined).mark_line().encode(
    alt.X('Date:T'),
    alt.Y('Return:Q',
          axis=alt.Axis(format='%'),
          scale=alt.Scale(type='log')),
    alt.Color('Strategy:O'),
    tooltip=['Date:T', 'Return:Q', 'Strategy:O']
)

alt.Chart(...)

In [10]:
ours_bad = np.array(nth_worst_stock_every_n_days(arr, 9, 3))
combined_return_bad = np.concatenate((ours_bad, theirs))
combined_strategy = ['Our Strategy']*3021 + ['S&P 500']*3021
combined_dates = np.concatenate((dates, dates))
combined_bad = pd.DataFrame({'Date': combined_dates,
                             'Strategy': combined_strategy,
                             'Return': combined_return_bad})

In [11]:
alt.Chart(combined_bad).mark_line().encode(
    alt.X('Date:T'),
    alt.Y('Return:Q', axis=alt.Axis(format='%')),
    alt.Color('Strategy:O'),
    tooltip=['Date:T', 'Return:Q', 'Strategy:O']
)

alt.Chart(...)